In [158]:
from datetime import (datetime, timedelta)
import itertools
import pytz

import numpy as np
from tables import *
import pandas as pd

from skyfield import api
from skyfield.sgp4lib import EarthSatellite
from skyfield.framelib import itrs

ts = api.load.timescale()

In [ ]:
h5file = open_file("/mnt/disk100/persist_home/meawoppl/export_test.h5", mode="r+")
tle_table = h5file.root.readout

# TODO: Precompute this. it takes a while to scrape the whole col...
all_ids = set(tle_table.cols.norad_id)
len(all_ids)

In [170]:
def tle_dataframe(records):
    return pd.DataFrame.from_records(records, columns=["epoch", "norad_id", "tle1", "tle2"])

def get_all_tles(norad_id: int):
    """
    Return a list of record types structured as
    (norad_id: int, epoch_time: float, tle_line_1: bytes, tle_line_2: bytes)
    """
    records = tle_table.read_where("norad_id=={}".format(norad_id))
    # Sort records based on time
    return tle_dataframe(sorted(records, key=lambda row: row[0]))


class TLEManager:
    def __init__(self, norad_id: int):
        self.records = get_all_tles(norad_id)
        
    def get_known_timespan(self):
        """
        Return datetimes corresponding to the first and last TLE epoch values
        in our record set
        """
        times = self.records.epoch.to_numpy()
        return times[0], times[-1]
    
    def get_compute_windows(self):
        """
        Return a dict which maps a tuple of epoch times, to a pair of TLE entries
        
        {
          (start_epoch, end_epoch) : (tle1, tle2)
        }
        """
        times = self.records.epoch.to_numpy()      
        ntle = times.shape[0]
        
        windows = {}
        for i in range(ntle):
            # TODO(meawoppl) gross, numpify me better
            
            start =  times[0] if i == 0 else (times[i-1] + times[i]) /2
            end   = times[-1] if i == ntle-1 else (times[i] + times[i+1])/2
        
            # NOTE(meawoppl) add holes to match AIS data somewhere in here....
            windows[(int(round(start)), int(round(end)))] = self.records.tle1[i], self.records.tle2[i]
        
        return windows
    
    def _epoch_to_julian(selt, time: float) -> float:
        return ts.from_datetime(datetime.utcfromtimestamp(time).replace(tzinfo=pytz.utc)).tt
    
    def compute_lat_long_dist(self, start_epoch: int, end_epoch: int, tle1, tle2):
        # Convert the start and end into julian
        start_time_j = self._epoch_to_julian(start_epoch)
        end_time_j = self._epoch_to_julian(end_epoch)
        julian_times = jds = ts.tt_jd(np.linspace(start_time_j, end_time_j, end_epoch - start_epoch))
        print(julian_times)
        sat = EarthSatellite(tle1.decode(), tle2.decode())
        return sat.at(jds).frame_latlon(itrs)


In [171]:
tlem = TLEManager(ids[-1])

In [172]:
windows = tlem.get_compute_windows()

In [175]:
for ((start, end), (tle1, tle2)) in windows.items():
    lats, longs, alts = tlem.compute_lat_long_dist(start, end, tle1, tle2)
    print(lats.degrees)
    print(longs.degrees)
    print(alts.m)


<Time tt=[2458388.0672243517 ... 2458389.051668796] len=85056>
[ 0.02597261  0.08743924  0.14890954 ... -1.8183541  -1.87813353
 -1.93790944]
[334.99890451 334.98514748 334.97138967 ... 160.43225688 160.41875314
 160.40524947]
[6983186.70483281 6983118.79985885 6983050.93911178 ... 7080991.26560404
 7081054.48233593 7081117.6315704 ]
<Time tt=[2458389.051668796 ... 2458391.122386389] len=178910>


MemoryError: Unable to allocate 938. MiB for an array with shape (687, 178910) and data type float64

In [ ]:
tlem.compute_lat_long_dist(w[0])

In [ ]:
def get_tracks_for_timerange(satellite: int, start: datetime, end: datetime):
    """
    Return a listing of active satellites and positions that spans `start` to `end`
    datetimes. The arrays returned will have `times` elements, which correspond to the
    number of integer minuted between the starting points of the two arrays.
    
    This returns an array:
    @returns np.array (times, 3)
    
    """
    pass
    
    